In [1]:
from pygam import PoissonGAM, s, f

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

import dill

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('weather_clean.csv', index_col=0)
data

,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,sunrise,sunset,preciptotal,...,DR,SH,FZ,MI,PR,BC,BL,VC,units,item_nbr
0,52,31,42,36,40,23,0,721,1714,0.05,...,False,False,True,False,False,False,False,False,0,1
1,52,31,42,36,40,23,0,721,1714,0.05,...,False,False,True,False,False,False,False,False,0,2
2,52,31,42,36,40,23,0,721,1714,0.05,...,False,False,True,False,False,False,False,False,0,3
3,52,31,42,36,40,23,0,721,1714,0.05,...,False,False,True,False,False,False,False,False,0,4
4,52,31,42,36,40,23,0,721,1714,0.05,...,False,False,True,False,False,False,False,False,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4369843,68,50,59,39,50,6,0,557,1719,0.00,...,False,False,False,False,False,False,False,False,0,107
4369844,68,50,59,39,50,6,0,557,1719,0.00,...,False,False,False,False,False,False,False,False,0,108
4369845,68,50,59,39,50,6,0,557,1719,0.00,...,False,False,False,False,False,False,False,False,0,109
4369846,68,50,59,39,50,6,0,557,1719,0.00,...,False,False,False,False,False,False,False,False,0,110


In [3]:
bool_cols = data.select_dtypes(include=['bool']).columns
data = data.drop(columns = bool_cols)
data

,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,sunrise,sunset,preciptotal,stnpressure,resultspeed,resultdir,avgspeed,city,units,item_nbr
0,52,31,42,36,40,23,0,721,1714,0.05,29.78,3.6,20,4.6,San Antonio,0,1
1,52,31,42,36,40,23,0,721,1714,0.05,29.78,3.6,20,4.6,San Antonio,0,2
2,52,31,42,36,40,23,0,721,1714,0.05,29.78,3.6,20,4.6,San Antonio,0,3
3,52,31,42,36,40,23,0,721,1714,0.05,29.78,3.6,20,4.6,San Antonio,0,4
4,52,31,42,36,40,23,0,721,1714,0.05,29.78,3.6,20,4.6,San Antonio,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4369843,68,50,59,39,50,6,0,557,1719,0.00,29.57,10.5,36,11.2,Jacksonville,0,107
4369844,68,50,59,39,50,6,0,557,1719,0.00,29.57,10.5,36,11.2,Jacksonville,0,108
4369845,68,50,59,39,50,6,0,557,1719,0.00,29.57,10.5,36,11.2,Jacksonville,0,109
4369846,68,50,59,39,50,6,0,557,1719,0.00,29.57,10.5,36,11.2,Jacksonville,0,110


In [5]:
cities = data.city.unique()

In [6]:
gam_data = {}
for city in cities:
    gam_data[city] = {}
    for i in range(1, 112):
        gam_data[city][i] = {}

In [8]:
mae = []
r2 = []

def pipeline_gridsearch(df, i, city):
    y = df['units']
    X = df.drop(columns=['units','item_nbr','city'])

    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)

    gam = PoissonGAM(max_iter=1000)
    gam.gridsearch(Xs, y, lam=[0.01, 0.1, 5], n_splines=[5, 10, 15], objective='AICc')
    ytp = gam.predict(Xs)

    raw_r2 = gam.statistics_['pseudo_r2']
    r2 = list(raw_r2.values())[0] if isinstance(raw_r2, dict) else raw_r2

    mae = mean_absolute_error(y, ytp)
    print(f"{city} #{i}: train MAE={mae:.2f}; train R²={r2:.3f}")

    return gam.coef_, gam.terms, mae, r2

for city in cities:
    for i in range(1, 112):
        df_i = data[(data.city == city) & (data['item_nbr']==i)]
        if df_i['units'].nunique() == 1 or df_i.empty:
            gam_data[city][i][0] = 0
            gam_data[city][i][1] = 0
        else:
            try:
                coefs, terms, _mae, _r2 = pipeline_gridsearch(df_i, i, city)
                gam_data[city][i][0] = coefs
                gam_data[city][i][1] = terms
                mae.append(_mae)
                r2.append(_r2)
            except Exception as e:
                print(e)
                gam_data[city][i][0] = 0
                gam_data[city][i][1] = 0

100% (9 of 9) |##########################| Elapsed Time: 0:00:03 Time:  0:00:030:00


San Antonio #9: train MAE=14.31; train R²=0.252


100% (9 of 9) |##########################| Elapsed Time: 0:00:03 Time:  0:00:030:00


San Antonio #28: train MAE=2.35; train R²=0.206


100% (9 of 9) |##########################| Elapsed Time: 0:00:09 Time:  0:00:090:01


...

Austin #36: train MAE=2.88; train R²=0.140


100% (9 of 9) |##########################| Elapsed Time: 0:00:08 Time:  0:00:080:00


Austin #103: train MAE=0.49; train R²=0.117


100% (9 of 9) |##########################| Elapsed Time: 0:00:11 Time:  0:00:110:01


Austin #111: train MAE=0.19; train R²=0.897


In [9]:
with open("gam_data.pkl", "wb") as f:
    dill.dump(gam_data, f)

In [10]:
with open("gam_data.pkl", "rb") as f:
    saved_data = dill.load(f)

# Восстанавливаем предсказание
def predict(X, terms, coef):
    return np.exp(terms.build_columns(X.reshape(1, -1)) @ coef)

{'San Antonio': {1: {0: 0, 1: 0},
  2: {0: 0, 1: 0},
  3: {0: 0, 1: 0},
  4: {0: 0, 1: 0},
  5: {0: 0, 1: 0},
  6: {0: 0, 1: 0},
  7: {0: 0, 1: 0},
  8: {0: 0, 1: 0},
  9: {0: array([-1.32977360e+01,  4.89609474e+00,  2.70922059e+00,  9.10454744e-01,
           8.93882940e-01,  4.75687580e-01,  8.16712384e-01,  5.18479471e-01,
           6.14840852e-02, -2.70557983e-01, -3.40751387e-01, -1.71570894e-01,
          -8.37945278e-01,  6.06302643e-01,  2.03512494e+00,  1.19393305e+01,
           9.96687193e-01, -2.32848434e-01, -1.22209873e+00, -1.07194006e+00,
          -1.46693982e+00, -1.18401818e+00, -1.25729139e+00, -1.49340457e+00,
          -1.43089475e+00, -1.47442568e+00, -1.30667453e+00, -1.92074298e+00,
          -4.63850362e-01,  5.93993846e-01, -4.32560076e+00, -4.54733388e+00,
           1.49407720e-01,  3.57817097e+00, -1.52035951e+00,  1.63835234e+00,
           5.73225895e+00,  1.78308266e+00, -1.86766861e+00, -1.45724068e+00,
          -2.16655579e+00, -1.36226085e+00, -8.

In [29]:
print(f"max mae: {round(np.max(mae), 2)} \nmin mae: {round(np.min(mae), 2)} \nmean mae: {round(np.mean(mae), 2)}")

max mae: 76.8 
min mae: 0.02 
mean mae: 6.66


In [31]:
print(f"max r2: {round(np.max(r2), 3)} \nmin r2: {round(np.min(r2), 3)} \nmean r2: {round(np.mean(r2), 3)}")

max r2: 0.926 
min r2: 0.01 
mean r2: 0.251


In [32]:
data_test.to_csv('test.csv')